In [ ]:
# importamos las dependencias
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    f1_score,
    make_scorer
)

In [ ]:
starter_df = pd.read_csv("../data/youtoxic_english_1000.csv")
starter_df = starter_df[["Text", "IsToxic"]]

starter_df.head(5)

In [ ]:
starter_df.info()

In [ ]:
count_hate = (starter_df["IsToxic"] == True).sum()
count_no_hate = (starter_df["IsToxic"] == False).sum()
print(f"Hay {count_hate} comentarios de odio")
print(f"Hay {count_no_hate} comentarios sin odio")

In [ ]:
new_df = pd.read_csv("../data/labeled_data.csv")

new_df.head()

In [ ]:
count = (new_df["hate_speech"] == 0).sum()
print(f"Hay {count} comentarios de odio en el nuevo dataset")

In [ ]:
nuevo_df = new_df[new_df["hate_speech"] == 0][["tweet", "hate_speech"]]
nuevo_df = nuevo_df.rename(columns={
    "tweet": "Text",
    "hate_speech": "IsToxic"
})
nuevo_df["IsToxic"] = nuevo_df["IsToxic"] == 0

nuevo_df = nuevo_df.sample(n=500, random_state=42)

nuevo_df.to_csv("../data/extra_data.csv", index=False)

print(nuevo_df.head(20))

In [ ]:
nuevo_df.info()

In [ ]:
extra_df = pd.read_csv("../data/extra_data.csv")
df = pd.concat([starter_df, extra_df], ignore_index=True)

In [ ]:
df.info()

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def limpiar_texto(texto):
    tokens = nltk.word_tokenize(str(texto).lower())
    tokens = [t for t in tokens if t.isalpha()]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return " ".join(tokens)

df['texto_limpio'] = df['Text'].apply(limpiar_texto)

vectorizer = TfidfVectorizer(
    max_features=170,
    min_df=35,
    max_df=0.6,
    ngram_range=(1, 2),
    sublinear_tf=True
)
X = vectorizer.fit_transform(df['texto_limpio'])
y = df['IsToxic'].astype(int)

modelo = LogisticRegression(C=0.055, max_iter=1500, class_weight="balanced")
f1_scorer = make_scorer(f1_score, pos_label=1)
scores = cross_val_score(modelo, X, y, cv=5, scoring=f1_scorer)

print(f"\n=== Validación cruzada (F1, clase tóxica) ===")
print(f"F1-score promedio: {scores.mean():.3f}")
print(f"Desviación estándar: {scores.std():.3f}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

modelo.fit(X_train, y_train)

y_train_pred = modelo.predict(X_train)
print("\n=== Evaluación en TRAIN ===")
print(confusion_matrix(y_train, y_train_pred))
print(classification_report(y_train, y_train_pred))

y_test_pred = modelo.predict(X_test)
print("\n=== Evaluación en TEST ===")
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

f1_train = f1_score(y_train, y_train_pred, pos_label=1)
f1_test = f1_score(y_test, y_test_pred, pos_label=1)

overfitting_f1_pct = ((f1_train - f1_test) / f1_train) * 100 if f1_train != 0 else 0.0
print(f"F1-score TRAIN (clase tóxica): {f1_train:.3f}")
print(f"F1-score TEST  (clase tóxica): {f1_test:.3f}")
print(f"Overfitting (basado en F1): {overfitting_f1_pct:.2f}%")

def predict_true_false(prob):
    return prob >= 0.5

def predecir_toxicidad(texto):
    texto_limpio = limpiar_texto(texto)
    texto_vect = vectorizer.transform([texto_limpio])
    prob = modelo.predict_proba(texto_vect)
    print(f"Probabilidades de cada clase [no tóxico, tóxico]: {prob}")
    prob_toxico = prob[0][1]
    print(f"Probabilidad de toxicidad: {prob_toxico:.3f}")
    prediccion_final = predict_true_false(prob_toxico)
    return {
        "prediccion": prediccion_final,
        "probabilidad_toxico": round(prob_toxico, 3)
    }

In [ ]:
print(predecir_toxicidad("stupid"))